In [6]:
import spacy
from spacy.pipeline import EntityRuler
import random

def preprocess_input(user_input):
    """Preprocess user input by converting to lowercase and stripping extra spaces."""
    return user_input.lower().strip()

# Load spaCy's pre-trained model
nlp = spacy.load("en_core_web_sm")

# Add EntityRuler to the pipeline
if "entity_ruler" not in nlp.pipe_names:
    ruler = nlp.add_pipe("entity_ruler", before="ner")
else:
    ruler = nlp.get_pipe("entity_ruler")

# Define custom patterns for medical terms
patterns = [
    {"label": "MEDICATION", "pattern": "Paracetamol"},
    {"label": "MEDICATION", "pattern": "Ibuprofen"},
    {"label": "MEDICATION", "pattern": "Aspirin"},
    {"label": "MEDICATION", "pattern": "Amoxicillin"},
    {"label": "MEDICATION", "pattern": "Penicillin"},
    {"label": "MEDICATION", "pattern": "Morphine"},
    {"label": "SYMPTOM", "pattern": "fever"},
    {"label": "SYMPTOM", "pattern": "headache"},
    {"label": "SYMPTOM", "pattern": "dizziness"},
    {"label": "SYMPTOM", "pattern": "nausea"},
    {"label": "SYMPTOM", "pattern": "fatigue"},
    {"label": "SYMPTOM", "pattern": "back pain"},
    {"label": "SYMPTOM", "pattern": "chest pain"},
    {"label": "SYMPTOM", "pattern": "shortness of breath"},
    {"label": "SYMPTOM", "pattern": "abdominal pain"},
    {"label": "SYMPTOM", "pattern": "vomiting"},
    {"label": "SYMPTOM", "pattern": "rashes"}
]

# Add patterns to the EntityRuler
ruler.add_patterns(patterns)

# Predefined rules for responses with variations
responses = {
    "SYMPTOM": {
        "fever": [
            "Stay hydrated and consider taking paracetamol if recommended by a doctor.",
            "Drink plenty of fluids and rest. If the fever persists, consult a doctor."
        ],
        "headache": [
            "Try to relax and consider over-the-counter pain relief. Consult a doctor if it worsens.",
            "Ensure you are well-hydrated and avoid stress. See a doctor if the headache persists."
        ],
        "nausea": [
            "Stay hydrated and eat light meals. Consult a doctor if nausea persists.",
            "Avoid heavy meals and drink ginger tea. If it continues, seek medical advice."
        ],
        "dizziness": [
            "Sit down and rest. If dizziness continues, consult a doctor.",
            "Ensure proper hydration and avoid sudden movements. Seek medical help if necessary."
        ],
        "fatigue": [
            "Ensure adequate rest and hydration. Consult a doctor if fatigue persists.",
            "Take breaks and avoid overexertion. If it doesn’t improve, see a doctor."
        ]
    },
    "MEDICATION": {
        "Paracetamol": [
            "Paracetamol is effective for reducing fever. Follow your doctor's dosage instructions.",
            "Use paracetamol as directed to manage pain or fever. Consult your doctor if symptoms persist."
        ],
        "Ibuprofen": [
            "Ibuprofen is often used for pain relief. Consult your doctor for detailed advice.",
            "Take ibuprofen with food to avoid stomach upset. Follow your doctor’s recommendations."
        ]
    }
}

# Predefined conditions database
conditions_database = {
    "diabetes": "Maintain a healthy diet, exercise regularly, and monitor blood sugar levels.",
    "hypertension": "Reduce salt intake, manage stress, and take prescribed medications.",
    "flu": "Rest, stay hydrated, and consider over-the-counter medication for symptom relief."
}

# Function to generate responses based on NER output
def generate_response(ner_output, keywords):
    response_parts = []

    # Handle symptoms
    if "SYMPTOM" in ner_output:
        for symptom in ner_output["SYMPTOM"]:
            advice = random.choice(responses["SYMPTOM"].get(symptom, [None]))
            if advice:
                response_parts.append(f"For {symptom}: {advice}")

    # Handle medications
    if "MEDICATION" in ner_output:
        for medication in ner_output["MEDICATION"]:
            advice = random.choice(responses["MEDICATION"].get(medication, [None]))
            if advice:
                response_parts.append(f"About {medication}: {advice}")

    # Keyword-based fallback
    if not response_parts:
        for keyword in keywords:
            if keyword in ner_output.get("text", "").lower():
                response_parts.append(f"It seems you mentioned '{keyword}'. Here's some advice: {conditions_database.get(keyword, 'Could you provide more details?')}.")
        if not response_parts:
            return "I'm sorry, I couldn't understand that. Could you describe your symptoms in more detail?"

    return " ".join(response_parts)

# Keywords for fallback response
keywords = list(conditions_database.keys())

# Chatbot interface
def chatbot():
    print("Welcome to the Medical Chatbot. Type 'exit' to end the session.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Goodbye! Stay healthy.")
            break

        preprocessed_input = preprocess_input(user_input)
        doc = nlp(preprocessed_input)

        ner_output = {"SYMPTOM": [], "MEDICATION": [], "text": user_input}
        for ent in doc.ents:
            if ent.label_ in ner_output:
                ner_output[ent.label_].append(ent.text)

        response = generate_response(ner_output, keywords)
        print(f"Bot: {response}")

# Test cases
if __name__ == "__main__":
    print("Running test cases...")
    test_inputs = [
        "I have a fever and headache.",
        "What should I do for nausea?",
        "I took Paracetamol for my fever.",
        "Tell me about hypertension.",
        "I'm feeling tired and dizzy."
    ]
    for test_input in test_inputs:
        preprocessed_input = preprocess_input(test_input)
        doc = nlp(preprocessed_input)

        ner_output = {"SYMPTOM": [], "MEDICATION": [], "text": test_input}
        for ent in doc.ents:
            if ent.label_ in ner_output:
                ner_output[ent.label_].append(ent.text)

        response = generate_response(ner_output, keywords)
        print(f"Input: {test_input}")
        print(f"Response: {response}\n")

    print("Launching chatbot...")
    chatbot()


Running test cases...
Input: I have a fever and headache.
Response: For fever: Drink plenty of fluids and rest. If the fever persists, consult a doctor. For headache: Ensure you are well-hydrated and avoid stress. See a doctor if the headache persists.

Input: What should I do for nausea?
Response: For nausea: Avoid heavy meals and drink ginger tea. If it continues, seek medical advice.

Input: I took Paracetamol for my fever.
Response: For fever: Drink plenty of fluids and rest. If the fever persists, consult a doctor.

Input: Tell me about hypertension.
Response: It seems you mentioned 'hypertension'. Here's some advice: Reduce salt intake, manage stress, and take prescribed medications..

Input: I'm feeling tired and dizzy.
Response: I'm sorry, I couldn't understand that. Could you describe your symptoms in more detail?

Launching chatbot...
Welcome to the Medical Chatbot. Type 'exit' to end the session.
You: I have diabetes
Bot: It seems you mentioned 'diabetes'. Here's some advice



```
Running test cases...
Input: I have a fever and headache.
Response: For fever: Drink plenty of fluids and rest. If the fever persists, consult a doctor. For headache: Ensure you are well-hydrated and avoid stress. See a doctor if the headache persists.

Input: What should I do for nausea?
Response: For nausea: Avoid heavy meals and drink ginger tea. If it continues, seek medical advice.

Input: I took Paracetamol for my fever.
Response: For fever: Drink plenty of fluids and rest. If the fever persists, consult a doctor.

Input: Tell me about hypertension.
Response: It seems you mentioned 'hypertension'. Here's some advice: Reduce salt intake, manage stress, and take prescribed medications..

Input: I'm feeling tired and dizzy.
Response: I'm sorry, I couldn't understand that. Could you describe your symptoms in more detail?

Launching chatbot...
Welcome to the Medical Chatbot. Type 'exit' to end the session.
You: I have diabetes
Bot: It seems you mentioned 'diabetes'. Here's some advice: Maintain a healthy diet, exercise regularly, and monitor blood sugar levels..
You: I have pain in my knees
Bot: I'm sorry, I couldn't understand that. Could you describe your symptoms in more detail?
You: I am experiencing dizzyness
Bot: I'm sorry, I couldn't understand that. Could you describe your symptoms in more detail?
You: I am experiencing dizziness
Bot: For dizziness: Ensure proper hydration and avoid sudden movements. Seek medical help if necessary.
You: exit
Goodbye! Stay healthy.```

